In [124]:
import networkx as nx 
import numpy as np
import matplotlib.pyplot as plt

from sklearn.datasets.base import Bunch

from regain.datasets.poisson import poisson_sampler
from regain.datasets.ising import ising_sampler

In [35]:
# Stessa struttura per tutte e tre le distribuzioni 

# scale-free

# erdo-reniy

#generate the base random graph we randomly add 10% of edges we randomly remove 10% of edges

In [107]:

def generate_multiple_class_dataset(n_dim_obs=10, n_edges=2, probability=0.2, n_classes=5, 
                                    _type='scale-free', random_state=None, plot=False):
    if _type=='scale-free':
        graph = nx.random_graphs.barabasi_albert_graph(n=n_dim_obs, m=n_edges, seed=random_state)
    else:
        graph = nx.random_graphs.erdos_renyi_graph(n=n_dim_obs, p=probability, seed=random_state)

    binaries = [nx.adjacency_matrix(graph).todense()]
    zeros = np.where(binaries[0]==0)
    nonzero= np.where(binaries[0]!=0)
    n_to_add = int(0.1*zeros[0].shape[0]) 
    n_to_remove = int(0.1*nonzero[0].shape[0]) 
    for i in range(n_classes-1):
        K_new = binaries[0].copy()
        to_add = np.random.choice(np.arange(zeros[0].shape[0]), n_to_add, replace=False)
        to_remove = np.random.choice(np.arange(nonzero[0].shape[0]), n_to_add, replace=False)
        for ta in to_add:
            a= np.random.choice([0,1], p=[0.2, 0.8])
            K_new[zeros[0][i], zeros[1][i]] = a
            K_new[zeros[1][i], zeros[0][i]] = a
        for ta in to_remove:
            a=np.random.choice([0,1], p=[0.8, 0.2])
            K_new[nonzero[0][i], nonzero[1][i]] = a
            K_new[nonzero[1][i], nonzero[0][i]] = a
        binaries.append(K_new)
    
    Ks_poisson = [b.copy().astype(np.float32) for b in binaries]  
    Ks_gaussian = [b.copy().astype(np.float32) for b in binaries]  
    A = np.random.rand(n_dim_obs, n_dim_obs)-0.5
    A = (A+A.T)/2
    for b, k in zip(binaries,Ks_gaussian):
        k[np.where(b!=0)] = A[np.where(b!=0)]
        np.fill_diagonal(k, 1)
    Ks_ising = [b.copy().astype(np.float32) for b in binaries]  
    A = np.random.rand(n_dim_obs, n_dim_obs)-0.5
    A = np.sign((A+A.T))
    for b, k in zip(binaries,Ks_ising):
        k[np.where(b!=0)] = A[np.where(b!=0)]
        np.fill_diagonal(k, 1)
   
    return binaries, Ks_poisson, Ks_gaussian, Ks_ising

In [108]:
binaries, poisson, gaussian, ising = generate_multiple_class_dataset(n_edges=1, _type='erdos-renyi')

In [125]:
def sample(n_dim_obs=10, n_samples=100, gaussian=None, ising=None, poisson=None,time_on_axis='first',):
    res= {}
    if gaussian is not None:
        sigmas = list(map(np.linalg.inv, gaussian))
        # map(normalize_matrix, sigmas)  # in place
        data = np.array(
        [
            np.random.multivariate_normal(
                np.zeros(n_dim_obs), sigma, size=n_samples) for sigma in sigmas
        ])

        X = np.vstack(data)
        y = np.repeat(range(len(sigmas)), n_samples).astype(int)

        if time_on_axis == "last":
            data = data.transpose(1, 2, 0)
        gaussian_res = Bunch(data=data, thetas=np.array(gaussian), X=X, y=y)
        res['gaussian'] = gaussian_res
    if poisson is not None:
        data = np.array([poisson_sampler(t, variances=np.zeros(n_dim_obs),
                               n_samples=n_samples) for t in poisson])
        X = np.vstack(data)
        y = np.repeat(range(len(poisson)), n_samples).astype(int)

        if time_on_axis == "last":
            data = data.transpose(1, 2, 0)
        poisson_res = Bunch(data=data, thetas=np.array(poisson), X=X, y=y)
        res['poisson'] = poisson_res
    if ising is not None:
        data = np.array([ising_sampler(t, np.zeros(n_dim_obs), n=n_samples, responses=[-1, 1])
                        for t in ising])
        X = np.vstack(data)
        y = np.repeat(range(len(poisson)), n_samples).astype(int)

        if time_on_axis == "last":
            data = data.transpose(1, 2, 0)
        ising_res = Bunch(data=data, thetas=np.array(ising), X=X, y=y)
        res['ising'] = ising_res
    return res

In [126]:
 res = sample(n_dim_obs=10, n_samples=100, gaussian=gaussian, ising=ising, poisson=poisson)

/home/vero/miniconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:9: RuntimeWarning: covariance is not positive-semidefinite.
  if __name__ == '__main__':



(10,)
(55,)
(100, 55)

(10,)
(55,)
(100, 55)

(10,)
(55,)
(100, 55)

(10,)
(55,)
(100, 55)

(10,)
(55,)
(100, 55)


In [128]:
def multi_class_kernel(n_classes=5, beta=1):
    kernel = np.ones((n_classes, n_classes))*beta
    np.fill_diagonal(kernel, 1)
    return kernel

In [129]:
kernel = multi_class_kernel(beta=0.5)

# Fitting

In [ ]:
from regain.covariance.kernel_time_graphical_lasso_ import KernelTimeGraphicalLasso
from regain.generalized_linear_mode